In [8]:
import pandas as pd
import wrds

In [9]:
conn = wrds.Connection(wrds_username="nberzins")

Loading library list...
Done


In [26]:
conn.describe_table(library='crsp', table='dsf')

Approximately 107663472 rows in crsp.dsf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,PERMNO
2,permco,True,INTEGER,PERMCO
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code Header
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


In [19]:
conn.describe_table(library='crsp', table='dsp500list')

Approximately 2064 rows in crsp.dsp500list.


,name,nullable,type,comment
0,permno,True,INTEGER,CRSP Permanent Number
1,start,True,DATE,Date when the stock included in S&P500 Index
2,ending,True,DATE,Date when the stock excluded from S&P500 Index


In [28]:
sp500 = conn.raw_sql("""
                        SELECT a.*, b.date, b.ret
                        FROM crsp.dsp500list as a, crsp.dsf as b
                        WHERE a.permno=b.permno
                            AND b.date >= a.start and b.date <= a.ending
                            AND b.date >= '01/01/2000'
                        ORDER BY date; 
                        """, date_cols=['start', 'ending', 'date'])

In [30]:
len(sp500["permno"])

3155303

In [41]:
dse = conn.raw_sql("""
                        SELECT comnam, namedt, nameendt,
                        permno, shrcd, exchcd
                        FROM crsp.dsenames
                        """, date_cols=['namedt', 'nameendt'])

In [42]:
dse.sample(5)

,comnam,namedt,nameendt,permno,shrcd,exchcd
101241,ENODIS PLC,2002-01-02,2004-06-09,88393,31,1
99939,BIO AQUA SYSTEMS INC,2001-02-01,2001-04-18,87833,11,2
42214,FALCON PRODUCTS INC,2002-01-02,2004-06-09,34121,11,1
71365,NATIONAL CITY BANCSHARES INC,1999-07-01,2000-05-21,77181,11,3
45952,HARRELL INTERNATIONAL INC,1972-12-14,1973-06-26,41022,10,3


In [43]:
sp500_full = pd.merge(sp500, dse, how='left', on='permno')

sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

sp500_full.head()

,permno,start,ending,date,ret,comnam,namedt,nameendt,shrcd,exchcd
2,12060,1957-03-01,2024-12-31,2000-01-03,-0.030695,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
11,70332,1997-07-28,2019-08-08,2000-01-03,-0.042125,ANADARKO PETROLEUM CORP,1986-10-02,2002-01-01,11,1
16,48485,1976-07-01,2006-03-31,2000-01-03,-0.02381,JEFFERSON PILOT CORP,1969-06-09,2001-08-23,11,1
23,18016,1957-03-01,2008-11-13,2000-01-03,-0.040359,HERCULES INC,1968-01-02,2001-08-23,11,1
29,91380,1987-01-08,2006-01-31,2000-01-03,0.015267,REEBOK INTERNATIONAL LTD,1986-12-03,2001-08-23,11,1


In [44]:
#sp500_full = sp500_full[sp500_full["start"]]

sp500_full[sp500_full["comnam"].isin(['GENERAL ELECTRIC CO'])]

,permno,start,ending,date,ret,comnam,namedt,nameendt,shrcd,exchcd
2,12060,1957-03-01,2024-12-31,2000-01-03,-0.030695,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
5200,12060,1957-03-01,2024-12-31,2000-01-04,-0.03875,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
8091,12060,1957-03-01,2024-12-31,2000-01-05,-0.005202,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
12818,12060,1957-03-01,2024-12-31,2000-01-06,0.015577,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
17521,12060,1957-03-01,2024-12-31,2000-01-07,0.038721,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
...,...,...,...,...,...,...,...,...,...,...
23933834,12060,1957-03-01,2024-12-31,2024-12-24,0.014614,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1
23938046,12060,1957-03-01,2024-12-31,2024-12-26,0.003849,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1
23941028,12060,1957-03-01,2024-12-31,2024-12-27,-0.010398,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1
23945492,12060,1957-03-01,2024-12-31,2024-12-30,-0.011642,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1


In [46]:
sp500_dict = {
    name: group[["date", "ret"]].reset_index(drop=True)
    for name, group in sp500_full.groupby("comnam")
}

In [48]:
#sp500_dict["GENERAL ELECTRIC CO"]

In [ ]:
#Need to Build Different Retun-Based Factors from Returns 

#e.g. Momentum, Volatility from past 30 days, market portfolio, baking/construction (responsive to IR), IR itself,
# others can be extracxted from French Fama ---- Shoot for 10 - 15 predictors for first iteration

# *Maybe find sentiment indices*

In [ ]:
#For next week -----------------------------

# Convert to weekly + Monthly data, too much noise in daily 
# Many large positions flip before market close to avoid sitting overnight 

#Focus on predictors and run standard models in ord

In [ ]:
# Next iteration would be to just include a ton of different predictors, 